## Diseño de viga doblemente armada

In [30]:
import numpy as np

phi = 0.9
fy = 4200 #kg/cm2
fc = 210 #kg/cm2
Es = 2040000 #kg/cm2

if fc <= 280:
    B1 = 0.85
elif fc > 280 and fc <= 560:
    B1 = 0.85 - (0.05*((fc-280)/70))
elif fc > 560:
    B1 = 0.65

eu = 0.003
ey = fy/Es
ety = 0.006 #ACI (For Tension controled members)

#cuantías
pmin = max((0.8*np.sqrt(fc)/fy), (14/fy))
pbal = 0.85*B1*(fc/fy)*(eu/(eu + ey)) 
pt = ((eu+ey)/(eu+ety)) * pbal
pmax = ((eu+ey)/(0.006+ey)) * pbal

print(f'pmin = {pmin}')
print(f'pbal = {pbal}')
print(f'pmax = {pmax}')
print(f'pt = {pt}')


Mu = 14 #Tonf*m

b = 0.2 #m
h = 0.4 #m
destribo = 0.01 #m
rec = 0.04 #m
R = pt*fy*(1 - (0.588*(pt*fy/fc)))*10 #*10 para pasar de kg/cm2 a Tonf/m2
varillas_asum_inf = 0.016 #m Asumimos una varillas de 16cm de diametro.
varillas_asum_sup = 0.016 #m Asumimos una varillas de 16cm de diametro.
d = h - rec - destribo - (varillas_asum_inf / 2)
dp = rec + destribo + (varillas_asum_sup/2)

#Determinación del acero As maximo
p = ((0.85*fc)/fy)*(1-np.sqrt(1-((2*R)/(0.85*fc*10))))
As_max = pmax*b*d #MODIFICABLE np.average((p,pmax))
Rmax = pmax*fy*(1 - (0.588*(pmax*fy/fc)))*10 #*10 para pasar de kg/cm2 a Tonf/m2
Mnmax = Rmax * b * (d ** 2)

if Mnmax < Mu:
    print('\nDiseñar como doblemente armada')
    print(f'\u03C6Mn < Mu, {Mnmax} < {Mu} Tonf*m')
else:
    print('\nSe puede diseñar como simplemente armada, simplemente aumentar la cuantía de acero seleccionada a pmax')
    print(f'\u03C6Mn > Mu, {Mnmax} > {Mu} Tonf*m')

Mu1 = Mnmax
As1 = As_max
Mu2 = (Mu/phi) - Mu1
As2 = Mu2/((fy*10)*(d-dp)) #Calculate As2 from Mu2 =𝜙As2fy(d−d′).


a = ((As1*10000)*fy)/(0.85*fc*(b*100))   #cm
c = a/B1
fs2 = min(((Es*0.003)*((c-dp)/c)),fy)
if fs2 >= fy: #Nominal Flexural Strength When the Compression Reinforcement Yields.
    print('\nRefuerzo a compresión sometido a esfuerzo de fluencia')
    Aps = As2 #m
    As = As1 + Aps #m
    phiMn = phi*((As1*(fy*10)*(d-((a/2)*0.01))) + (As2*(fy*10)*(d-dp)))
elif fs2 < fy: #Nominal Flexural Strength When the Compression Reinforcement Does Not Yield.
    print('\nRefuerzo a compresión no sometido a esfuerzo de fluencia')
    Aps = As2*(fy/fs2)
    a1 = 0.85*(fc)*(b*100)*B1
    b1 = ((Es*0.003)*(Aps*10000)) - ((As1*10000)*fy)
    c = (-b1 + (np.sqrt((b1**2)+(24480*a1*Aps*dp))))/(2*a1)
    a = c*B1
    fs2 = 0.003*Es*(1-((dp*100)/c))
    phiMn = phi*((0.85*(fc*10)*(a*0.01)*b*(d-(a*0.01/2))) + (Aps*(fs2*10)*(d - dp)))


pp = (b*h)/As2 #m
p1 = p - pp 
if p1 > pmax:
    print('\nLa sección se encuentra en zona de transición, \u03C6 < 0.9 para Mu1 y \u03C6 = 0.9 para Mu2')
    print(f'\u03C6Mn = {phiMn} Tonf*m')
else:
    print('\nLa sección se encuentra controlada a tensión, \u03C6 = 0.9 para Mu1 y Mu2')
    print(f'\u03C6Mn = {phiMn} Tonf*m')





pmin = 0.0033333333333333335
pbal = 0.021422965116279065
pmax = 0.013447992700729926
pt = 0.012041666666666664

Diseñar como doblemente armada
φMn < Mu, 11.123065830129574 < 14 Tonf*m

Refuerzo a compresión sometido a esfuerzo de fluencia

La sección se encuentra controlada a tensión, φ = 0.9 para Mu1 y Mu2
φMn = 13.99924745928338 Tonf*m


In [31]:
import Varillas as Var

tolerancia = 1.5
Varillas_sup = Var.Varillas(Aps*10000,tolerancia) #Tomar en cuenta que el previo valor de As está en m2, la función sólo aplica con cm2.
Varillas_inf = Var.Varillas(As1*10000,tolerancia) #Tomar en cuenta que el previo valor de As está en m2, la función sólo aplica con cm2.

print('\nVarillas Superiores')
print(f'\nAs superior = {Aps*10000} cm2')
for i in range(len(Varillas_sup)):
    print(f"{i+1} -  {Varillas_sup[i, 0]} \u03C6 {Varillas_sup[i, 1]}, Area = {Varillas_sup[i, 2]} cm2")


Varillas_comb_sup = Var.Varillas_comb(Aps*10000,tolerancia) #Tomar en cuenta que el previo valor de As está en m2, la función sólo aplica con cm2.
print('\nVarillas combinadas superiores')
for i in range(len(Varillas_comb_sup)):
    print(f"{i+1} -  {Varillas_comb_sup[i, 0]} \u03C6 {Varillas_comb_sup[i, 1]} & {Varillas_comb_sup[i, 2]} \u03C6 {Varillas_comb_sup[i, 3]}, Area = {Varillas_comb_sup[i, 4]} cm2")

print('\nVarillas Inferiores')
print(f'\nAs inferior = {As1*10000} cm2')
for i in range(len(Varillas_sup)):
    print(f"{i+1} -  {Varillas_inf[i, 0]} \u03C6 {Varillas_inf[i, 1]}, Area = {Varillas_inf[i, 2]} cm2")


Varillas_comb_inf = Var.Varillas_comb(As1*10000,tolerancia) #Tomar en cuenta que el previo valor de As está en m2, la función sólo aplica con cm2.
print('\nVarillas combinadas inferiores')
for i in range(len(Varillas_comb_inf)):
    print(f"{i+1} -  {Varillas_comb_inf[i, 0]} \u03C6 {Varillas_comb_inf[i, 1]} & {Varillas_comb_inf[i, 2]} \u03C6 {Varillas_comb_inf[i, 3]}, Area = {Varillas_comb_inf[i, 4]} cm2")



Varillas Superiores

As superior = 3.7160376638380117 cm2
1 -  3.0 φ 14.0, Area = 4.62 cm2
2 -  2.0 φ 16.0, Area = 4.02 cm2
3 -  2.0 φ 18.0, Area = 5.09 cm2
4 -  1.0 φ 22.0, Area = 3.8 cm2
5 -  1.0 φ 25.0, Area = 4.91 cm2

Varillas combinadas superiores
1 -  1.0 φ 14.0 & 1.0 φ 18.0, Area = 4.08 cm2
2 -  1.0 φ 14.0 & 1.0 φ 20.0, Area = 4.68 cm2
3 -  1.0 φ 16.0 & 1.0 φ 20.0, Area = 5.15 cm2

Varillas Inferiores

As inferior = 9.198427007299271 cm2
1 -  6.0 φ 14.0, Area = 9.24 cm2
2 -  5.0 φ 16.0, Area = 10.05 cm2
3 -  4.0 φ 18.0, Area = 10.18 cm2
4 -  3.0 φ 20.0, Area = 9.42 cm2
5 -  2.0 φ 25.0, Area = 9.82 cm2

Varillas combinadas inferiores
1 -  2.0 φ 14.0 & 2.0 φ 20.0, Area = 9.36 cm2
2 -  3.0 φ 14.0 & 2.0 φ 18.0, Area = 9.71 cm2
3 -  4.0 φ 14.0 & 1.0 φ 20.0, Area = 9.3 cm2
4 -  5.0 φ 14.0 & 1.0 φ 18.0, Area = 10.24 cm2
5 -  1.0 φ 16.0 & 2.0 φ 22.0, Area = 9.61 cm2
6 -  2.0 φ 16.0 & 2.0 φ 20.0, Area = 10.3 cm2
7 -  3.0 φ 16.0 & 1.0 φ 22.0, Area = 9.83 cm2
8 -  1.0 φ 18.0 & 2.0 φ 22.0